变量&副作用

In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
count_variable = tf.get_variable('count',[])
sess = tf.Session()
print(sess.run(count_variable))

主要有两种将值放入变量的方法：初始化器和 tf.assign()。我们先看看 tf.assign()：

In [ ]:
count_variable_1 = tf.get_variable("count_variable_1", [])
zero_node = tf.constant(0.)
assign_node = tf.assign(count_variable_1, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable_1))

tf.assign(target, value) 是具备一些独特属性：

恒等运算。tf.assign(target, value) 不做任何有趣的运算，通常与 value 相等。

副作用。当计算「流经」assign_node 时，副作用发生在图中的其他节点上。此时，副作用是用存储在 zero_node 中的值替换 count_variable 的值。

非依赖边。即使 count_variable 节点和 assign_node 在图中是相连的，但它们彼此独立。这意味着计算任一节点时，计算不会通过边回流。然而，assign_node 依赖于 zero_node，它需要知道分配了什么。

初始化器

In [ ]:
const_init_node = tf.constant_initializer(0.)
count_variable_2 = tf.get_variable("count_variable_2", [], initializer=const_init_node)
sess = tf.Session()
print(sess.run([count_variable]))

问题出现在会话和图之间的分离。我们已将 get_variable 的 initializer 属性设置为指向 const_init_node，但它只是在图中的节点之间添加了一个新的连接。我们还没有做任何解决异常根源的事：与变量节点（存储在会话中，而不是计算图中）相关联的内存仍然设置为「null」。我们需要通过会话使 const_init_node 去更新变量。

In [10]:
const_init_node = tf.constant_initializer(0.)
count_variable_3 = tf.get_variable("count_variable_3", [], initializer=const_init_node)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(count_variable))

1.1983985


变量共享

优化器

In [ ]:
import tensorflow as tf
# build the graph## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
init = tf.global_variables_initializer()
# then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
y_guess = m * x + b

loss = tf.square(y - y_guess)
# finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)
# start the session
sess = tf.Session()
sess.run(init)
# perform the training loop*import* random
# set up problem
true_m = random.random()
true_b = random.random()
*for* update_i * in* range(100000):
    # (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b

    # (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _ = sess.run([loss, train_op], feed_dict={
                        input_placeholder: input_data, output_placeholder: output_data})
    *print* update_i, _loss
# finally, print out the values we learned for our two variables*print* "True parameters: m=%.4f, b=%.4f" % (true_m, true_b)*print* "Learned parameters: m=%.4f, b=%.4f" % tuple(sess.run([m, b]))

用tf.Print调试

有关 tf.Print 一个重要且有点微妙的点：打印是一个副作用。像所有其他副作用一样，只要在计算流经 tf.Print 节点时才会进行打印。如果 tf.Print 节点不在计算路径上，则不会打印任何内容。特别的是，即使 tf.Print 节点正在复制的原始节点位于计算路径上，但 tf.Print 节点本身可能不在。请注意这个问题！当这种情况发生时（总会发生的），如果你没有明确地找到问题所在，它会让你感到十分沮丧。一般来说，最好在创建要复制的节点后，立即创建你的 tf.Print 节点。